In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
pd.set_option('display.max_columns', None)

In [2]:
dataPath = "data/"

pairs = pd.read_csv(dataPath + "relation.csv")
users = pd.read_csv(dataPath + "customers.csv", index_col="id")
items = pd.read_csv(dataPath + "products.csv", index_col="id")


/tmp/ipykernel_867/1774659839.py:4: DtypeWarning: Columns (7,9,10,11,12,13,17,18,21,26,27,28,30,36) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv(dataPath + "customers.csv", index_col="id")
/tmp/ipykernel_867/1774659839.py:5: DtypeWarning: Columns (20,32,34,39) have mixed types. Specify dtype option on import or set low_memory=False.
  items = pd.read_csv(dataPath + "products.csv", index_col="id")


### Compañia 15

In [3]:
df_pairs = pairs[ pairs.company_id == 15]#.copy()
df_users = users[ users.company_id == 15]#.copy()
df_items = items[ items.company_id == 15]#.copy()

In [4]:
# Comprobacion de que no haya usuarios ni productos duplicados

assert df_users.index.duplicated().any()==False
assert df_items.index.duplicated().any()==False

### Quitar usuarios que interaccionan pero no estan contemplados en la tabla usuarios (unos 575)

In [5]:
usuarios_que_interaccionan_pero_no_en_tabla_usuarios = set(df_pairs.customer_id) - set(df_users.index)
len(usuarios_que_interaccionan_pero_no_en_tabla_usuarios)

575

In [6]:
df_pairs = df_pairs[ ~df_pairs.customer_id.isin(usuarios_que_interaccionan_pero_no_en_tabla_usuarios) ]
len(df_pairs)

59167

### Quitar productos que interaccionan pero no estan contemplados en la tabla productos (unos 162)

In [7]:
productos_que_interaccionan_pero_no_en_tabla_productos = set(df_pairs.product_id) - set(df_items.index)
len(productos_que_interaccionan_pero_no_en_tabla_productos)

162

In [8]:
df_pairs = df_pairs[ ~df_pairs.product_id.isin(productos_que_interaccionan_pero_no_en_tabla_productos) ]
len(df_pairs)

58913

# <center> Fin código limepieza datos
---
# <center> Inicio código Sistema de recomendación

In [9]:
from recommenderSystem2 import RecSys2
r = RecSys2()

In [10]:
r.entrenar_modelo(df_pairs, df_users, df_items)

Leyendo datos...
Entrenando modelo...


100%|███████████████████████████████████████████████████████████████████████| 921/921 [00:15<00:00, 61.28it/s]


tensor(5.4664, grad_fn=<MseLossBackward0>)


100%|███████████████████████████████████████████████████████████████████████| 921/921 [00:15<00:00, 59.76it/s]

tensor(2.9257, grad_fn=<MseLossBackward0>)
Generando embeddings...
Terminado


In [11]:
r.activeUser_ids[:5], r.newUser_ids[:5]

(tensor([21724, 21726, 21728, 21730, 21731]),
 tensor([21725, 21727, 21729, 21742, 21744]))

In [12]:
r.activeItem_ids[:5], r.newItem_ids[:5]

(tensor([28863, 28864, 28865, 28866, 28867]),
 tensor([28871, 28873, 28879, 28881, 28936]))

In [19]:
r.recomendar(query_user_ids=[21724, 21726], searchOn_act_users=True, limit=0.85)

tensor([221309,  21726,  21724, 537630, 519512, 532329, 195560,  22905,  39015,
         54316,  52099, 156105, 174775,  53041,  39604,  38596, 164492, 230107,
         32594, 123663, 161798,  51199, 529564, 310159,  35344, 147240,  40519,
        219748,  25339,  47018,  26448,  51410, 532309,  35827,  47915,  30990,
         29510, 160024, 170994, 169060, 474534, 507316, 533895,  49815, 160664,
         27026,  30128, 529567, 141682, 153470,  35703,  52303,  40875, 518481,
         21956,  27302, 529171, 470086,  47839,  23976,  25804,  29267, 347783,
         23428,  33654,  56539,  56073, 144134,  55353,  25803,  39990, 148599,
        161435, 153377, 170360, 147527, 146083, 537382, 141624, 535224, 518815,
        286147, 518688, 144044, 534033, 476082,  40118,  23966, 518814,  27109,
        534291, 519028, 147695, 159203, 151272, 174458])

In [17]:
r.recomendar(query_item_ids=[28863, 28871], searchOn_act_users=True, limit=0.7)

tensor([ 55195,  56453,  51657, 157076,  34875,  50350,  38741,  25184, 171685,
         48278,  34912, 123924,  25527])

In [15]:
r.newUser_embs

tensor([[-0.3789, -0.2819, -0.1088,  ..., -0.1622,  0.0772,  0.1856],
        [-0.3789, -0.2819, -0.1088,  ..., -0.1622,  0.0772,  0.1856],
        [-0.3789, -0.2819, -0.1088,  ..., -0.1622,  0.0772,  0.1856],
        ...,
        [-0.3789, -0.2819, -0.1088,  ..., -0.1622,  0.0772,  0.1856],
        [-0.3114, -0.2811, -0.0444,  ..., -0.2418,  0.0261,  0.3358],
        [-0.3614, -0.2409, -0.1065,  ..., -0.1766,  0.0508,  0.2750]])

In [16]:
r.activeUser_embs

tensor([[-0.3027, -0.3158, -0.0994,  ..., -0.2051,  0.0772,  0.1898],
        [-0.3414, -0.2335,  0.0355,  ..., -0.1534,  0.0730,  0.1757],
        [-0.4750, -0.1962, -0.0757,  ..., -0.1128, -0.3266,  0.1292],
        ...,
        [-0.4414, -0.2616, -0.0134,  ..., -0.1528,  0.1147, -0.0774],
        [-0.2688, -0.4214, -0.0383,  ..., -0.2170,  0.0283,  0.2902],
        [-0.3199, -0.2381,  0.5726,  ..., -0.1369,  0.0655,  0.0968]])

In [29]:
r.activeItem_embs

tensor([[ 0.0702,  0.1729, -0.0071,  ..., -0.0731, -0.2270,  0.0632],
        [ 0.0703,  0.1734, -0.0069,  ..., -0.0736, -0.2288,  0.0627],
        [ 0.0697,  0.1731, -0.0068,  ..., -0.0734, -0.2280,  0.0622],
        ...,
        [ 0.0650,  0.1755, -0.0098,  ..., -0.0737, -0.2267,  0.0589],
        [ 0.0648,  0.1754, -0.0053,  ..., -0.0737, -0.2263,  0.0587],
        [ 0.0189,  0.1969,  0.0089,  ..., -0.0771, -0.2106,  0.0263]])